In [19]:
!pip install beautifulsoup4

You are using pip version 18.1, however version 20.2.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [130]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [58]:
reference = {
    "dcinside" : {
        "url":" https://gall.dcinside.com/board/lists",
        "params": {
            "id": "superidea",
            "list_num": "100",
            "sort_type": "N",
            "page": "1",
        },
        "headers" : {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36 Edg/86.0.622.38"},
    }
}

In [59]:
dc = reference["dcinside"]
print(dc)
res = requests.get(dc["url"], params=dc["params"], headers=dc["headers"])
res.status_code

{'url': ' https://gall.dcinside.com/board/lists', 'params': {'id': 'superidea', 'list_num': '100', 'sort_type': 'N', 'page': '1'}, 'headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36 Edg/86.0.622.38'}}


200

In [60]:
soup = BeautifulSoup(res.content, 'html.parser')

In [75]:
for row in soup.find('tbody').find_all('tr')[1:]:
    print(row.find_all("td"))
    break

[<td class="gall_num">232104</td>, <td class="gall_tit ub-word">
<a href="/board/view/?id=superidea&amp;no=232104&amp;_rk=96k&amp;list_num=100&amp;page=1"><em class="icon_img icon_toprecomimg"></em><strong>[자갤]</strong> 오늘 새벽라이딩 끝</a>
<a class="reply_numbox" href="https://gall.dcinside.com/board/view/?id=superidea&amp;no=232104&amp;t=cv&amp;list_num=100&amp;page=1&amp;_rk=glb"><span class="reply_num">[15]</span></a> </td>, <td class="gall_writer ub-writer" data-ip="" data-loc="list" data-nick="JohannSebastianBach" data-uid="minseihun">
<span class="nickname in" title="JohannSebastianBach"><em>JohannSebastianBach</em></span><a class="writer_nikcon"><img alt="갤로그로 이동합니다." border="0" height="11" onclick="window.open('//gallog.dcinside.com/minseihun');" src="https://nstatic.dcinside.com/dc/w/images/nik.gif" style="margin-left:2px;cursor:pointer;" title="minseih** : 갤로그로 이동합니다." width="12"/></a> </td>, <td class="gall_date" title="2020-10-29 10:14:04">10.29</td>, <td class="gall_count">685<

In [136]:
class DcinsideCrawler:
    INFOS = {
        'base_url': 'https://gall.dcinside.com',
        'lists': '/board/lists',
        'params': {
            "id": "superidea",
            "list_num": "100",
            "sort_type": "N",
            "page": "1",
        },
        'headers': {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36 Edg/86.0.622.38"
        }
    }
    
    def crawl(self):
        results = []
        res = requests.get(
            self.INFOS['base_url'] + self.INFOS['lists'], 
            params=self.INFOS['params'], 
            headers=self.INFOS['headers']
        )
        soup = BeautifulSoup(res.content, 'html.parser')
        rows = soup.find('tbody').find_all('tr')[1:] # first one is survey
        for row in rows:
            cells = row.find_all("td")
            url = self.INFOS['base_url'] + cells[1].find("a")["href"]
            title = list(cells[1].find("a").children)[-1].strip()
            reply_num = cells[1].find("a", {"class": "reply_numbox"}).text[1:-1]
            date = datetime.strptime(cells[3]["title"], "%Y-%m-%d %H:%M:%S")
            view = cells[4].text
            results.append({
                "src": "디씨인사이드",
                "url": url, 
                "title": title, 
                "reply_num": reply_num, 
                "date": date,
                "view": view
            })
        return results

crawler = DcinsideCrawler()
crawler.crawl()
                        

[{'src': '디씨인사이드',
  'url': 'https://gall.dcinside.com/board/view/?id=superidea&no=232104&_rk=96k&list_num=100&page=1',
  'title': '오늘 새벽라이딩 끝',
  'reply_num': '15',
  'date': datetime.datetime(2020, 10, 29, 10, 14, 4),
  'view': '929'},
 {'src': '디씨인사이드',
  'url': 'https://gall.dcinside.com/board/view/?id=superidea&no=232103&_rk=tDL&list_num=100&page=1',
  'title': '만들었는데 자랑하고 싶어서...',
  'reply_num': '53',
  'date': datetime.datetime(2020, 10, 28, 16, 59, 54),
  'view': '4829'},
 {'src': '디씨인사이드',
  'url': 'https://gall.dcinside.com/board/view/?id=superidea&no=232102&_rk=CFQ&list_num=100&page=1',
  'title': '덥스텝 연대기 (3) 덥스텝의 의의와 UK에서의 발전과정',
  'reply_num': '49',
  'date': datetime.datetime(2020, 10, 28, 4, 31, 8),
  'view': '685'},
 {'src': '디씨인사이드',
  'url': 'https://gall.dcinside.com/board/view/?id=superidea&no=232101&_rk=Hc2&list_num=100&page=1',
  'title': '[코스프레 대회] 정장 스가 코스프레+추가',
  'reply_num': '34',
  'date': datetime.datetime(2020, 10, 17, 0, 24, 3),
  'view': '2029'},
 {'src